# Data Cleaning & Preprocessing (ETL)
## Business Intelligence Project – Sales Analytics and Waste Reduction in Bakery Production


### Project Overview
This notebook documents the **Data Cleaning and Preprocessing (ETL)** phase of the bakery BI project.

The bakery sells **perishable products**, making accurate daily sales analysis critical for:
- Understanding demand patterns  
- Identifying seasonality  
- Inferring waste and slow-moving product risks  





### Dataset Description
The dataset contains daily transaction-level data with the following columns:
- `date`: Transaction date  
- `time`: Transaction time  
- `ticket_number`: Transaction ID  
- `article`: Product name (French)  
- `quantity`: Quantity sold  
- `unit_price`: Price per unit in euros

Time coverage: **2021-01-01 to 2022-09-30**


## 1. Import Libraries

In [102]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
sns.set_style("whitegrid")


## 2. Load and Inspect Data

Load dataset

In [103]:
df = pd.read_csv("Bakery sales.csv")

Initial inspection



In [104]:
df.head()

,Unnamed: 0,date,time,ticket_number,article,Quantity,unit_price
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"
2,4,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €"
3,5,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €"
4,8,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €"



Dataset structure

In [105]:
df.shape, df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     234005 non-null  int64  
 1   date           234005 non-null  object 
 2   time           234005 non-null  object 
 3   ticket_number  234005 non-null  float64
 4   article        234005 non-null  object 
 5   Quantity       234005 non-null  float64
 6   unit_price     234005 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 12.5+ MB


((234005, 7), None)

 Basic descriptive statistics

In [106]:
df.describe()


,Unnamed: 0,ticket_number,Quantity
count,234005.000000,234005.000000,234005.000000
mean,255205.035854,219201.258738,1.538377
std,147501.625992,40053.223896,1.289603
min,0.000000,150040.000000,-200.000000
25%,127979.000000,184754.000000,1.000000
50%,254573.000000,218807.000000,1.000000
75%,382911.000000,253927.000000,2.000000
max,511395.000000,288913.000000,200.000000


## 3. Data Quality Checks

Missing values

In [107]:
df.isna().sum()


,0
Unnamed: 0,0
date,0
time,0
ticket_number,0
article,0
Quantity,0
unit_price,0



Duplicate rows

In [108]:
df.duplicated().sum()


np.int64(0)

Clean and convert unit_price to numeric

In [109]:
df['unit_price'] = df['unit_price'].astype(str).str.replace('€', '').str.replace(',', '.').astype(float)

Invalid values checks

In [110]:
invalid_quantity = (df['Quantity'] <= 0).sum()
invalid_price = (df['unit_price'] <= 0).sum()

invalid_quantity, invalid_price

(np.int64(1295), np.int64(32))

Convert `ticket_number` to integer

In [111]:
df['ticket_number'] = df['ticket_number'].astype(int)

Convert `quantity` to integer

In [112]:
df['Quantity'] = df['Quantity'].astype(int)


## 4. Data Cleaning


 Standardize column names first

In [113]:
df.columns = df.columns.str.lower().str.strip()

Remove invalid records (quantity and unit_price)

In [114]:
df = df[(df['quantity'] > 0) & (df['unit_price'] > 0)]

Drop duplicates

In [115]:
df = df.drop_duplicates()

Convert date and time, with explicit format

In [116]:
df['date'] = pd.to_datetime(df['date'])
df['time'] = pd.to_datetime(df['time'], format='%H:%M')
df['time'] = df['time'].dt.strftime('%H:%M')

## 5. Feature Engineering


The following features are created to support BI analysis:
- **Revenue**: Core financial metric  
- **Calendar features**: Identify trends, seasonality, and weekly behavior  
- **Time-of-day features**: Useful for staffing, production planning, and waste reduction  


Revenue

In [117]:
df['revenue'] = df['quantity'] * df['unit_price']

Time-based features

In [118]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['weekday'] = df['date'].dt.day_name()
df['is_weekend'] = df['weekday'].isin(['Saturday', 'Sunday']).astype(int)

Hour extraction

In [119]:
df['hour'] = pd.to_datetime(df['time'].astype(str)).dt.hour

/tmp/ipython-input-3610405445.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['hour'] = pd.to_datetime(df['time'].astype(str)).dt.hour


 Time of day categorization

In [120]:
def time_of_day(hour):
    if hour < 12:
        return 'Morning'
    elif hour < 17:
        return 'Afternoon'
    else:
        return 'Evening'

df['time_of_day'] = df['hour'].apply(time_of_day)


## 6. Aggregation for BI

Daily sales aggregation

In [121]:
daily_sales = df.groupby('date').agg(
    total_units_sold=('quantity', 'sum'),
    total_revenue=('revenue', 'sum')
).reset_index()

daily_sales.head()


,date,total_units_sold,total_revenue
0,2021-01-02,580,987.85
1,2021-01-03,565,1015.20
2,2021-01-04,315,461.90
3,2021-01-05,312,526.10
4,2021-01-07,310,544.00


 Daily sales per product:

In [122]:

daily_product_sales = df.groupby(['date', 'article']).agg(
    daily_units=('quantity', 'sum'),
    daily_revenue=('revenue', 'sum')
).reset_index()

daily_product_sales.head()


,date,article,daily_units,daily_revenue
0,2021-01-02,BAGUETTE,46,41.4
1,2021-01-02,BANETTE,40,42.0
2,2021-01-02,BANETTINE,6,3.6
3,2021-01-02,BOULE 200G,6,6.6
4,2021-01-02,BOULE 400G,11,16.5


## 7. Export Clean Data


Export datasets

In [123]:
df.to_csv("clean_bakery_sales.csv", index=False)
daily_sales.to_csv("daily_sales.csv", index=False)
daily_product_sales.to_csv("daily_product_sales.csv", index=False)